In [66]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.18.2


In [67]:
# !ls
import os
train_path = 'training/'
test_path = 'testing/'

def get_data(folder_path, data):
    filelist = os.listdir(folder_path)

    for file_name in filelist:
        if file_name.endswith('.csv'):
            with open(folder_path + file_name, 'r') as f:
                r = csv.reader(f)
                data.append([tuple(row) for row in r])

import csv
train_sents = list()
test_sents = list()
get_data(train_path, train_sents)
get_data(test_path, test_sents)




Untitled.ipynb   items.py         price_1.crfsuite testing
__init__.py      middlewares.py   settings.py      training
__init__.pyc     pipelines.py     settings.pyc     whole_data
back_up_csv_data pipelines.pyc    spiders


In [68]:
# len(data)
# import random
# random.shuffle(data)

# train_sents = data[:50]
# test_sents = data[50:71]


In [69]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

In [70]:
sent2features(train_sents[0])[0]

['bias',
 'word.lower=for',
 'word[-3:]=For',
 'word[-2:]=or',
 'word.isupper=False',
 'word.istitle=True',
 'word.isdigit=False',
 'postag=IN',
 'postag[:2]=IN',
 'BOS',
 '+1:word.lower=sale',
 '+1:word.istitle=True',
 '+1:word.isupper=False',
 '+1:postag=NNP',
 '+1:postag[:2]=NN']

In [71]:
# %%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 60.7 ms, sys: 8.85 ms, total: 69.5 ms
Wall time: 77.1 ms


In [72]:
# %%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 95.1 ms, sys: 4.65 ms, total: 99.7 ms
Wall time: 108 ms


In [73]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [74]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [75]:
# %%time
trainer.train('price_1.crfsuite')

CPU times: user 273 ms, sys: 6.45 ms, total: 280 ms
Wall time: 305 ms


In [76]:
trainer.logparser.last_iteration

{'active_features': 51,
 'error_norm': 1.228869,
 'feature_norm': 5.569129,
 'linesearch_step': 1.0,
 'linesearch_trials': 1,
 'loss': 52.93176,
 'num': 50,
 'scores': {},
 'time': 0.004}

In [77]:
print len(trainer.logparser.iterations), trainer.logparser.iterations[-1]

50 {'loss': 52.93176, 'error_norm': 1.228869, 'linesearch_trials': 1, 'active_features': 51, 'num': 50, 'time': 0.004, 'scores': {}, 'linesearch_step': 1.0, 'feature_norm': 5.569129}


In [78]:
tagger = pycrfsuite.Tagger()

tagger.open('price_1.crfsuite')

In [79]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

I have a super low mileage 07 Vento Colt that i wan na sell . This bike was bought for me as a gift but i dont know much about it . It only has 311miles on it . I dont know anything about bikes and i cant seem to keep it running . But when it did run , it actually ran really good . I drained all the gas out of it . It has never been layed down , it is clean title , i just want to get rid of it . It literally has been sitting for 2 years . $ 2000 obo . Shoot me an offer No i do not need help selling ! ! And no BS about paypal , cashier check , no dumbass deposits nothing like that !
('Predicted:', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O P O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O')
('Correct:  ', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O

In [80]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [81]:
# %%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 18.5 ms, sys: 769 µs, total: 19.3 ms
Wall time: 20.4 ms


In [82]:
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          P       0.92      0.88      0.90        26

avg / total       0.92      0.88      0.90        26

